# Mekan Detayları

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
import random
import re

def setup_driver():
    """Chrome driver'ı ayarla"""
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-extensions")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
    
    # Bot tespitini engellemek için
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    return driver

def clean_filename(text):
    """Dosya adı için güvenli string oluştur"""
    # Türkçe karakterleri İngilizce karşılıkları ile değiştir
    replacements = {
        'ç': 'c', 'ğ': 'g', 'ı': 'i', 'ö': 'o', 'ş': 's', 'ü': 'u',
        'Ç': 'C', 'Ğ': 'G', 'İ': 'I', 'Ö': 'O', 'Ş': 'S', 'Ü': 'U'
    }
    
    for turkish, english in replacements.items():
        text = text.replace(turkish, english)
    
    # Sadece harf, rakam ve boşluk bırak
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Boşlukları alt çizgi ile değiştir
    text = text.replace(' ', '_')
    # Çoklu alt çizgileri tek alt çizgi yap
    text = re.sub(r'_+', '_', text)
    # Baş ve sondaki alt çizgileri kaldır
    text = text.strip('_')
    
    return text.lower()

def search_places(driver, search_term):
    """Belirtilen terimi Google Maps'ta ara"""
    try:
        # Google Maps'e git
        driver.get("https://www.google.com/maps")
        
        # Sayfanın yüklenmesini bekle
        wait = WebDriverWait(driver, 15)
        
        # Arama kutusunu bekle ve bul
        search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
        time.sleep(random.uniform(2, 4))
        
        # Arama yap
        search_box.clear()
        search_box.send_keys(search_term)
        time.sleep(random.uniform(1, 2))
        search_box.send_keys(Keys.ENTER)
        
        # Sonuçların yüklenmesini bekle
        time.sleep(random.uniform(5, 8))
        
        return True
        
    except TimeoutException:
        print("❌ Arama kutusu bulunamadı veya sayfa yüklenemedi")
        return False

def scroll_and_load_results(driver, max_results=None):
    """Sonuçları scroll yaparak yükle - Tüm sonuçları yükler"""
    try:
        # Sonuçlar panelini bul
        wait = WebDriverWait(driver, 10)
        
        # Google Maps'ta sonuçlar paneli için farklı selector'lar
        panel_selectors = [
            '[role="main"] [role="region"]',
            '.siAUzd',
            '[data-value="Sonuçlar"]',
            '.section-layout .section-scrollbox',
            '[aria-label*="sonuç"]'
        ]
        
        results_panel = None
        for selector in panel_selectors:
            try:
                results_panel = driver.find_element(By.CSS_SELECTOR, selector)
                print(f"✅ Sonuçlar paneli bulundu: {selector}")
                break
            except NoSuchElementException:
                continue
        
        if not results_panel:
            print("❌ Sonuçlar paneli bulunamadı, sayfa scrollu yapılacak")
            # Eğer panel bulunamazsa, sayfanın kendisini scroll et
            results_panel = driver.find_element(By.TAG_NAME, "body")
        
        print(f"📜 Scroll işlemi başlıyor... (Tüm mekanlar için)")
        previous_count = 0
        no_change_count = 0
        max_no_change = 10  # Değişim olmadığında daha fazla deneme yapmak için artırıldı
        scroll_attempt = 0
        
        unique_links = set()
        
        while True:
            scroll_attempt += 1
            # Mevcut sonuç sayısını al
            current_cards = driver.find_elements(By.CSS_SELECTOR, 'a[data-value="Directions"]')
            if not current_cards:
                current_cards = driver.find_elements(By.XPATH, '//a[contains(@class, "hfpxzc")]')
            
            # Tekrar eden linkleri filtrele
            filtered_cards = []
            for card in current_cards:
                try:
                    link = card.get_attribute("href")
                    if link and "/maps/place/" in link and link not in unique_links:
                        unique_links.add(link)
                        filtered_cards.append(card)
                except:
                    continue
            
            current_count = len(unique_links)
            
            print(f"🔄 Scroll {scroll_attempt}: {current_count} benzersiz mekan bulundu")
            
            # Değişim kontrolü
            if current_count == previous_count:
                no_change_count += 1
                print(f"⏳ Değişim yok ({no_change_count}/{max_no_change})")
            else:
                no_change_count = 0
                previous_count = current_count
            
            # Belirli sayıda üst üste değişim yoksa bitir
            if no_change_count >= max_no_change:
                print(f"✅ Daha fazla yeni mekan bulunamadı, toplam: {current_count}")
                break
            
            # Scroll yap - hem JavaScript hem de Keys kullan
            try:
                # JavaScript scroll
                driver.execute_script(
                    "arguments[0].scrollBy(0, 500);", 
                    results_panel
                )
                time.sleep(1)
                
                # Alternatif scroll yöntemi
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(random.uniform(1, 3))
                
                # Son elemana git ve aşağı tuşuna bas
                if current_cards:
                    last_card = current_cards[-1]
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", last_card)
                    time.sleep(1)
                    
                    # Son karta odaklan ve aşağı tuşlarına bas
                    try:
                        last_card.click()
                        time.sleep(0.5)
                        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
                        time.sleep(1)
                    except:
                        pass
                
            except Exception as e:
                print(f"⚠️ Scroll hatası: {e}")
            
            # "Daha fazla sonuç" butonu varsa tıkla
            try:
                more_button = driver.find_element(By.XPATH, "//*[contains(text(), 'Daha fazla')]")
                driver.execute_script("arguments[0].click();", more_button)
                print("🔄 'Daha fazla' butonuna tıklandı")
                time.sleep(3)
            except:
                pass
        
        print(f"✅ Scroll işlemi tamamlandı. Toplam benzersiz mekan: {current_count}")
        return True
        
    except Exception as e:
        print(f"❌ Scroll işleminde hata: {str(e)}")
        return False

def extract_place_details(driver, place_url):
    """Bir mekanın detay sayfasından bilgileri çıkar"""
    details = {
        'adres': 'Bilgi yok',
        'puan': 'Bilgi yok',
        'yorum_sayisi': 'Bilgi yok',
        'kategori': 'Bilgi yok',
        'fiyat_araligi': 'Bilgi yok'
    }
    
    try:
        # Yeni sekmede aç
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(place_url)
        
        # Sayfa yüklenmesini bekle
        time.sleep(random.uniform(4, 6))
        
        # DEBUG: Sayfanın yüklendiğini kontrol et
        print(f"🔍 Sayfa yüklendi: {driver.title[:50]}...")
        
        # YORUM SAYISI VE PUANLARI BULMAK İÇİN TÜM METİN ELEMENTLERİNİ TARA
        try:
            # Önce tüm sayfayı tara ve potansiyel puan/yorum içeren elementleri bul
            all_texts = driver.find_elements(By.XPATH, "//*[text()]")
            page_text = ""
            
            for element in all_texts:
                try:
                    text = element.text.strip()
                    if text:
                        page_text += " " + text
                except:
                    pass
            
            # Parantez içinde sayı formatını ara (2.334) veya (2,334)
            parenthesis_matches = re.findall(r'\(([0-9.,]+)\)', page_text)
            if parenthesis_matches:
                for match in parenthesis_matches:
                    # Sayı olup olmadığını kontrol et
                    cleaned_match = match.replace('.', '').replace(',', '')
                    if cleaned_match.isdigit() and int(cleaned_match) > 10:  # Genelde yorum sayısı 10'dan büyük
                        details['yorum_sayisi'] = cleaned_match
                        print(f"💬 Sayfa metninden yorum sayısı: {match} -> {details['yorum_sayisi']}")
                        break
            
            # Puanı ara (4.1 veya 4,1 formatında)
            rating_matches = re.findall(r'(\d[.,]\d)(?:\s|$|\()', page_text)
            if rating_matches:
                for match in rating_matches:
                    rating_value = match.replace(',', '.')
                    if 1.0 <= float(rating_value) <= 5.0:
                        details['puan'] = rating_value
                        print(f"⭐ Sayfa metninden puan: {match} -> {details['puan']}")
                        break
        except Exception as e:
            print(f"⚠️ Sayfa tarama hatası: {e}")
        
        # Adres bilgisi
        try:
            address_selectors = [
                '[data-item-id="address"] .Io6YTe',
                '.Io6YTe[data-value="Adres"]',
                '[data-value="Address"] .Io6YTe',
                '.rogA2c .Io6YTe',
                '[data-item-id="address"] span',
                '.section-info-line .Io6YTe'
            ]
            
            for selector in address_selectors:
                try:
                    address_element = driver.find_element(By.CSS_SELECTOR, selector)
                    details['adres'] = address_element.text.strip()
                    if details['adres']:
                        print(f"📍 Adres bulundu: {details['adres'][:30]}...")
                        break
                except:
                    continue
                    
            # Alternatif adres arama
            if details['adres'] == 'Bilgi yok':
                try:
                    address_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'Io6YTe')]")
                    for elem in address_elements:
                        text = elem.text.strip()
                        if text and ('Mah' in text or 'Cd' in text or 'Blv' in text or 'Sk' in text):
                            details['adres'] = text
                            print(f"📍 Alternatif adres: {details['adres'][:30]}...")
                            break
                except:
                    pass
        except:
            pass
        
        # Yorum sayısı ve puan zaten bulunduysa diğer yöntemleri atla
        if details['yorum_sayisi'] == 'Bilgi yok' or details['puan'] == 'Bilgi yok':
            # Puan ve yorum sayısı - Güncellenmiş selector'lar
            try:
                # Puan için yeni selector'lar
                rating_selectors = [
                    'div.F7nice span[aria-hidden="true"]',
                    '.ceNzKf[aria-label*="yıldız"]',
                    '.ceNzKf[aria-label*="star"]',
                    'span.ceNzKf',
                    '.jANrlb .fontDisplayLarge',
                    '.section-star-display',
                    'span.fontDisplayLarge'
                ]
                
                for selector in rating_selectors:
                    try:
                        rating_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        for rating_element in rating_elements:
                            rating_text = rating_element.text.strip()
                            # Puan kontrolü - 1.0 ile 5.0 arasında olmalı
                            if rating_text and re.match(r'^[1-5][.,]\d$', rating_text):
                                details['puan'] = rating_text.replace(',', '.')
                                print(f"⭐ Puan bulundu: {rating_text} -> {details['puan']}")
                                break
                        if details['puan'] != 'Bilgi yok':
                            break
                    except:
                        continue
                
                # Yorum sayısı için güncellenmiş selector'lar ve parantez içini alma
                review_selectors = [
                    'div.F7nice .bC3Nkc',
                    '.HHrUdb.fontTitleSmall',
                    '.ceNzKf + .HHrUdb',
                    '.jANrlb .fontBodyMedium',
                    '.section-rating-term',
                    'button[aria-label*="yorum"]',
                    'button[aria-label*="review"]',
                    'span[aria-label*="review"]',
                    'span.fontBodyMedium',
                    'span.UY7F9'
                ]
                
                for selector in review_selectors:
                    try:
                        review_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        for review_element in review_elements:
                            review_text = review_element.text.strip()
                            
                            # Parantez içindeki sayıyı bul (2.334) gibi
                            parenthesis_match = re.search(r'\(([0-9.,]+)\)', review_text)
                            if parenthesis_match:
                                review_count = parenthesis_match.group(1).replace('.', '').replace(',', '')
                                if review_count.isdigit() and int(review_count) > 10:
                                    details['yorum_sayisi'] = review_count
                                    print(f"💬 Parantez içi yorum sayısı: {parenthesis_match.group(1)} -> {details['yorum_sayisi']}")
                                    break
                            
                            # Eğer parantez içinde değilse normal sayıları kontrol et
                            numbers = re.findall(r'[\d.,]+', review_text)
                            if numbers:
                                for number in numbers:
                                    cleaned_number = number.replace('.', '').replace(',', '')
                                    if cleaned_number.isdigit() and int(cleaned_number) > 10:
                                        details['yorum_sayisi'] = cleaned_number
                                        print(f"💬 Yorum sayısı bulundu: {number} -> {details['yorum_sayisi']}")
                                        break
                        
                        if details['yorum_sayisi'] != 'Bilgi yok':
                            break
                    except:
                        continue
                        
                # Alternatif yorum arama - aria-label'dan
                if details['yorum_sayisi'] == 'Bilgi yok':
                    try:
                        review_elements = driver.find_elements(By.XPATH, "//*[@aria-label]")
                        for element in review_elements:
                            try:
                                aria_label = element.get_attribute('aria-label')
                                if not aria_label:
                                    continue
                                    
                                # Parantez içindeki sayıyı kontrol et
                                parenthesis_match = re.search(r'\(([0-9.,]+)\)', aria_label)
                                if parenthesis_match:
                                    review_count = parenthesis_match.group(1).replace('.', '').replace(',', '')
                                    if review_count.isdigit() and int(review_count) > 10:
                                        details['yorum_sayisi'] = review_count
                                        print(f"💬 Aria-label parantez içi yorum: {parenthesis_match.group(1)} -> {details['yorum_sayisi']}")
                                        break
                                
                                # Normal sayıları kontrol et
                                numbers = re.findall(r'[\d.,]+', aria_label)
                                for number in numbers:
                                    cleaned_number = number.replace('.', '').replace(',', '')
                                    if cleaned_number.isdigit() and int(cleaned_number) > 10:
                                        details['yorum_sayisi'] = cleaned_number
                                        print(f"💬 Aria-label yorum sayısı: {number} -> {details['yorum_sayisi']}")
                                        break
                            except:
                                continue
                    except:
                        pass
            except Exception as e:
                print(f"⚠️ Puan/yorum bulma hatası: {e}")
        
        # Kategori/tür bilgisi - Güncellenmiş
        try:
            category_selectors = [
                '.DkEaL',
                '.mgr77e .DkEaL',
                '.fontBodyMedium .DkEaL',
                '.section-hero-header-title-subtitle .DkEaL',
                '.section-rating-term .DkEaL'
            ]
            
            for selector in category_selectors:
                try:
                    category_element = driver.find_element(By.CSS_SELECTOR, selector)
                    category_text = category_element.text.strip()
                    if category_text and category_text != details['puan']:
                        details['kategori'] = category_text
                        print(f"🏷️ Kategori bulundu: {details['kategori']}")
                        break
                except:
                    continue
        except:
            pass
        
        # Fiyat aralığı - class="mgr77e" altındaki tüm bilgileri al
        try:
            # mgr77e class'ı altındaki tüm fiyat bilgilerini ara
            mgr77e_elements = driver.find_elements(By.CSS_SELECTOR, '.mgr77e')
            for mgr_element in mgr77e_elements:
                try:
                    # İçindeki tüm metni al
                    full_text = mgr_element.text.strip()
                    if full_text:
                        # Sadece fiyat sembollerini içeren kısımları filtrele
                        if '₺' in full_text:
                            # ₺₺₺ formatını veya ₺1.000+ gibi formatları içeren satırı bul
                            price_lines = []
                            for line in full_text.split('\n'):
                                if '₺' in line:
                                    price_lines.append(line.strip())
                            
                            if price_lines:
                                details['fiyat_araligi'] = price_lines[0]  # İlk fiyat bilgisini al
                                print(f"💰 mgr77e fiyat bilgisi: {details['fiyat_araligi']}")
                                break
                except:
                    continue
            
            # Eğer mgr77e içinde bulunamadıysa diğer yöntemleri dene
            if details['fiyat_araligi'] == 'Bilgi yok':
                # Fiyat için farklı yaklaşımlar - TL sembolü için
                price_selectors = [
                    'span[aria-label*="Pahalı"]',
                    'span[aria-label*="Expensive"]',
                    'span[aria-label*="Price"]',
                    'span[aria-label*="Fiyat"]',
                    '.fontBodyMedium span[aria-label*="$"]',
                    '.fontBodyMedium span[aria-label*="₺"]',
                    'span.LEs8xd'
                ]
                
                for selector in price_selectors:
                    try:
                        price_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        for price_element in price_elements:
                            aria_label = price_element.get_attribute('aria-label') or ''
                            text_content = price_element.text.strip()
                            
                            # TL sembolü (₺) kontrolü
                            if '₺' in text_content:
                                details['fiyat_araligi'] = text_content
                                print(f"💰 Fiyat (TL): {text_content}")
                                break
                            
                            # Dolar sembolü kontrolü
                            if '$' in text_content:
                                details['fiyat_araligi'] = text_content
                                print(f"💰 Fiyat (Dolar): {text_content}")
                                break
                            
                            # Aria label'da fiyat kontrolü
                            if 'Pahalı' in aria_label or 'Expensive' in aria_label or 'Price' in aria_label or 'Fiyat' in aria_label:
                                # Pahalılık derecesini tahmin et
                                if 'çok pahalı' in aria_label.lower() or 'very expensive' in aria_label.lower():
                                    details['fiyat_araligi'] = '₺₺₺₺'
                                elif 'orta' in aria_label.lower() or 'moderate' in aria_label.lower():
                                    details['fiyat_araligi'] = '₺₺'
                                elif 'ucuz' in aria_label.lower() or 'inexpensive' in aria_label.lower():
                                    details['fiyat_araligi'] = '₺'
                                else:
                                    details['fiyat_araligi'] = '₺₺₺'
                                
                                print(f"💰 Fiyat (tahmin): {aria_label} -> {details['fiyat_araligi']}")
                                break
                                
                        if details['fiyat_araligi'] != 'Bilgi yok':
                            break
                    except:
                        continue
                
                # Alternatif: Tüm span'leri kontrol et
                if details['fiyat_araligi'] == 'Bilgi yok':
                    try:
                        all_spans = driver.find_elements(By.TAG_NAME, 'span')
                        for span in all_spans:
                            text = span.text.strip()
                            if '₺' in text and len(text) <= 20:  # Daha uzun metinleri de kabul et
                                details['fiyat_araligi'] = text
                                print(f"💰 Alternatif fiyat (TL): {text}")
                                break
                            elif '$' in text and len(text) <= 20:
                                details['fiyat_araligi'] = text
                                print(f"💰 Alternatif fiyat (Dolar): {text}")
                                break
                    except:
                        pass
        except Exception as e:
            print(f"⚠️ Fiyat bulma hatası: {e}")
        
        # Sekmeyi kapat ve ana sekmeye dön
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        return details
        
    except Exception as e:
        print(f"⚠️ Detay çıkarma hatası: {e}")
        # Hata durumunda ana sekmeye dön
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass
        return details

def extract_data(driver, max_results=None):
    """Restoran verilerini çıkar - Tüm mekanlar için detaylı bilgilerle"""
    data = []
    
    try:
        # Tüm sonuçları al
        wait = WebDriverWait(driver, 5)
        
        # Farklı selector'ları dene
        all_cards = []
        selectors = [
            'a[data-value="Directions"]',  # Yol tarifi linkleri
            'a.hfpxzc',                    # Temel kart linkleri
            '[role="article"] a',          # Makale içindeki linkler
            '.section-result a'            # Sonuç bölümündeki linkler
        ]
        
        for selector in selectors:
            cards = driver.find_elements(By.CSS_SELECTOR, selector)
            if cards:
                all_cards.extend(cards)
                print(f"✅ {len(cards)} kart bulundu ({selector})")
        
        # Tekrarları kaldır
        unique_cards = []
        seen_links = set()
        
        for card in all_cards:
            try:
                link = card.get_attribute("href")
                if link and link not in seen_links and "/maps/place/" in link:
                    unique_cards.append(card)
                    seen_links.add(link)
            except:
                continue
        
        print(f"📊 Benzersiz {len(unique_cards)} kart bulundu")
        
        # Tüm mekanları işle
        process_cards = unique_cards
        print(f"🔍 Tüm {len(process_cards)} mekan için detaylı bilgiler toplanıyor...")
        
        for i, card in enumerate(process_cards):
            try:
                # Temel bilgileri al
                name = card.get_attribute("aria-label")
                link = card.get_attribute("href")
                
                # Alternatif isim alma yöntemleri
                if not name:
                    try:
                        name = card.find_element(By.CSS_SELECTOR, '[role="img"]').get_attribute("aria-label")
                    except:
                        try:
                            name = card.find_element(By.TAG_NAME, "span").text
                        except:
                            name = f"Mekan {i+1}"
                
                if name and link and "/maps/place/" in link:
                    # İsmi temizle
                    name = name.strip()
                    if "·" in name:
                        name = name.split("·")[0].strip()
                    
                    print(f"🔍 {i+1}/{len(process_cards)}: {name[:30]}...")
                    
                    # Detaylı bilgileri al
                    details = extract_place_details(driver, link)
                    
                    # Verileri birleştir
                    place_data = {
                        "Restoran_Adi": name,
                        "Adres": details['adres'],
                        "Puan": details['puan'],
                        "Yorum_Sayisi": details['yorum_sayisi'],
                        "Kategori": details['kategori'],
                        "Fiyat_Araligi": details['fiyat_araligi'],
                        "Google_Maps_Link": link
                    }
                    
                    data.append(place_data)
                    
                    # Progress göster
                    if (i + 1) % 5 == 0:
                        print(f"✅ {i+1} kayıt tamamlandı...")
                    
                    # Rate limiting
                    time.sleep(random.uniform(1, 2))
                    
            except Exception as e:
                print(f"⚠️ Kart {i+1} işlenirken hata: {str(e)}")
                continue
        
        print(f"✅ Toplam {len(data)} detaylı veri çıkarıldı")
        return data
        
    except Exception as e:
        print(f"❌ Veri çıkarma hatası: {str(e)}")
        return data

def save_to_csv(data, search_term):
    """Verileri CSV'ye kaydet"""
    if not data:
        print("❌ Kaydedilecek veri yok")
        return
    
    # Dosya adını oluştur
    clean_search = clean_filename(search_term)
    filename = f"google_maps_{clean_search}_detayli.csv"
    
    try:
        # DataFrame oluştur
        df = pd.DataFrame(data)
        
        # Veri tiplerini düzelt
        for i, row in df.iterrows():
            # Puanları string olarak formatla (Excel'de tarih olarak görünmesini engellemek için)
            try:
                if row['Puan'] != 'Bilgi yok':
                    # Virgülü noktaya çevir ve formatla
                    puan_str = str(row['Puan']).replace(',', '.')
                    # String olarak formatlanmış değeri kaydet (örn: "4.5")
                    df.at[i, 'Puan'] = f"'{float(puan_str):.1f}"
            except Exception as e:
                print(f"Puan dönüştürme hatası: {e}, değer: {row['Puan']}")
                df.at[i, 'Puan'] = "'Bilgi yok"
                
            # Yorum sayılarını sayıya dönüştür
            try:
                if row['Yorum_Sayisi'] != 'Bilgi yok':
                    # Nokta ve virgülleri kaldır
                    yorum_str = str(row['Yorum_Sayisi']).replace('.', '').replace(',', '')
                    if yorum_str.isdigit():
                        df.at[i, 'Yorum_Sayisi'] = int(yorum_str)
            except Exception as e:
                print(f"Yorum sayısı dönüştürme hatası: {e}, değer: {row['Yorum_Sayisi']}")
        
        # Tekrar eden kayıtları kaldır
        df = df.drop_duplicates(subset=['Restoran_Adi'])
        
        # Puan sütununun metin olarak CSV'ye yazılmasını sağla
        # to_numeric kullanmak yerine string olarak bırak
        
        # CSV'ye kaydet - UTF-8 BOM ile Türkçe karakterler için
        # float_format kaldırıldı çünkü artık değerler string
        df.to_csv(filename, index=False, encoding='utf-8-sig', sep=';')
        
        print(f"\n📈 Detaylı Sonuçlar:")
        print(f"Toplam {len(df)} yer bulundu")
        print(f"💾 Veriler '{filename}' dosyasına kaydedildi")
        
        # İlk 5 kaydı detaylı göster
        if len(df) > 0:
            print(f"\n📋 İlk {min(5, len(df))} kayıt özeti:")
            for idx, row in df.head(5).iterrows():
                puan_str = row['Puan'].replace("'", "") if isinstance(row['Puan'], str) else "Bilgi yok"
                print(f"\n{idx+1}. {row['Restoran_Adi']}")
                print(f"   📍 Adres: {row['Adres']}")
                print(f"   ⭐ Puan: {puan_str} ({row['Yorum_Sayisi']} yorum)")
                print(f"   🏷️ Kategori: {row['Kategori']}")
                print(f"   💰 Fiyat: {row['Fiyat_Araligi']}")
        
        return filename
        
    except Exception as e:
        print(f"❌ Dosya kaydetme hatası: {str(e)}")
        return None

def main():
    """Ana fonksiyon"""
    driver = None
    
    try:
        # Kullanıcıdan arama terimi al
        print("🔍 Google Maps Detaylı Veri Çıkarıcı")
        print("-" * 40)
        search_term = input("Ne aramak istiyorsunuz? (örn: restoranlar Kadıköy, kafe Beşiktaş): ").strip()
        
        if not search_term:
            print("❌ Arama terimi boş olamaz!")
            return
        
        print(f"\n🚀 Tüm mekanlar için detaylı arama başlatılıyor: '{search_term}'")
        print("⚠️  Bu işlem her mekan için detay sayfasını ziyaret edeceği için biraz zaman alacak...")
        print("-" * 60)
        
        print("🌐 Chrome tarayıcı başlatılıyor...")
        driver = setup_driver()
        
        print("🔍 Google Maps'ta arama yapılıyor...")
        if not search_places(driver, search_term):
            return
        
        print("📜 Tüm mekanlar yükleniyor...")
        scroll_and_load_results(driver)
        
        print("📊 Detaylı veriler çıkarılıyor...")
        data = extract_data(driver)
        
        if data:
            save_to_csv(data, search_term)
        else:
            print("❌ Hiç veri bulunamadı")
            
    except KeyboardInterrupt:
        print("\n⏹️ İşlem kullanıcı tarafından durduruldu")
        
    except Exception as e:
        print(f"❌ Genel hata: {str(e)}")
        
    finally:
        if driver:
            print("\n🔄 Tarayıcı kapatılıyor...")
            try:
                driver.quit()
                print("✅ İşlem tamamlandı")
            except:
                print("⚠️ Tarayıcı zaten kapalı")

if __name__ == "__main__":
    main()

🔍 Google Maps Detaylı Veri Çıkarıcı
----------------------------------------

🚀 Tüm mekanlar için detaylı arama başlatılıyor: 'alsancak izmir restoranlar'
⚠️  Bu işlem her mekan için detay sayfasını ziyaret edeceği için biraz zaman alacak...
------------------------------------------------------------
🌐 Chrome tarayıcı başlatılıyor...
🔍 Google Maps'ta arama yapılıyor...
📜 Tüm mekanlar yükleniyor...
✅ Sonuçlar paneli bulundu: [role="main"] [role="region"]
📜 Scroll işlemi başlıyor... (Tüm mekanlar için)
🔄 Scroll 1: 6 benzersiz mekan bulundu
🔄 'Daha fazla' butonuna tıklandı
🔄 Scroll 2: 12 benzersiz mekan bulundu
🔄 'Daha fazla' butonuna tıklandı
🔄 Scroll 3: 18 benzersiz mekan bulundu
🔄 'Daha fazla' butonuna tıklandı
🔄 Scroll 4: 20 benzersiz mekan bulundu
🔄 'Daha fazla' butonuna tıklandı
🔄 Scroll 5: 20 benzersiz mekan bulundu
⏳ Değişim yok (1/10)
🔄 'Daha fazla' butonuna tıklandı
🔄 Scroll 6: 26 benzersiz mekan bulundu
🔄 'Daha fazla' butonuna tıklandı
🔄 Scroll 7: 26 benzersiz mekan bulundu
⏳ 

# Yorum Toplama

In [ ]:
print("Hello World")

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
import random
import re
import os

def setup_driver():
    """Chrome driver'ı ayarla"""
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-extensions")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
    
    # Bot tespitini engellemek için
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    return driver

def clean_filename(text):
    """Dosya adı için güvenli string oluştur"""
    # Türkçe karakterleri İngilizce karşılıkları ile değiştir
    replacements = {
        'ç': 'c', 'ğ': 'g', 'ı': 'i', 'ö': 'o', 'ş': 's', 'ü': 'u',
        'Ç': 'C', 'Ğ': 'G', 'İ': 'I', 'Ö': 'O', 'Ş': 'S', 'Ü': 'U'
    }
    
    for turkish, english in replacements.items():
        text = text.replace(turkish, english)
    
    # Sadece harf, rakam ve boşluk bırak
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Boşlukları alt çizgi ile değiştir
    text = text.replace(' ', '_')
    # Çoklu alt çizgileri tek alt çizgi yap
    text = re.sub(r'_+', '_', text)
    # Baş ve sondaki alt çizgileri kaldır
    text = text.strip('_')
    
    return text.lower()

def read_csv_places(csv_filename):
    """CSV dosyasından mekan linklerini oku"""
    try:
        # CSV dosyasını oku
        df = pd.read_csv(csv_filename, sep=';', encoding='utf-8-sig')
        
        places = []
        for index, row in df.iterrows():
            place_info = {
                'name': row.get('Restoran_Adi', f'Mekan_{index+1}'),
                'link': row.get('Google_Maps_Link', '')
            }
            
            # Link kontrolü
            if place_info['link'] and '/maps/place/' in place_info['link']:
                places.append(place_info)
            else:
                print(f"⚠️ Geçersiz link atlandı: {place_info['name']}")
        
        print(f"✅ CSV'den {len(places)} geçerli mekan linki okundu")
        return places
        
    except Exception as e:
        print(f"❌ CSV okuma hatası: {e}")
        return []

def extract_reviews(driver, place_url, place_name, max_reviews=100):
    """Bir mekanın yorumlarını çıkar ve CSV'ye kaydet"""
    reviews = []
    try:
        print(f"\n🔍 '{place_name}' için yorumlar toplanıyor (maksimum {max_reviews} yorum)...")
        
        # Yeni sekmede aç
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(place_url)
        
        # Sayfa yüklenmesini bekle
        time.sleep(random.uniform(4, 6))
        
        # Yorumlar kısmına tıkla
        try:
            # Yorumlar butonunu bul
            review_buttons = driver.find_elements(By.CSS_SELECTOR, "div.Gpq6kf.NlVald")
            review_button = None
            
            # Yorumlar butonunu bul
            for button in review_buttons:
                try:
                    button_text = button.text.strip()
                    if "Yorumlar" in button_text or "Reviews" in button_text or "yorum" in button_text.lower() or "review" in button_text.lower():
                        review_button = button
                        print(f"✅ Yorumlar butonu bulundu: '{button_text}'")
                        break
                except:
                    continue
            
            if review_button:
                print("✅ Yorumlar butonuna tıklanıyor...")
                driver.execute_script("arguments[0].click();", review_button)
                time.sleep(random.uniform(3, 5))
            else:
                print("⚠️ Yorumlar butonu bulunamadı, alternatif yöntem deneniyor...")
                # Alternatif yöntem 1: reviews tab'ına gitmeyi dene
                tabs = driver.find_elements(By.CSS_SELECTOR, "button[role='tab']")
                tab_found = False
                for tab in tabs:
                    try:
                        tab_text = tab.text.strip()
                        if "Yorumlar" in tab_text or "Reviews" in tab_text or "yorum" in tab_text.lower() or "review" in tab_text.lower():
                            driver.execute_script("arguments[0].click();", tab)
                            print(f"✅ Yorumlar sekmesi bulundu: '{tab_text}'")
                            tab_found = True
                            time.sleep(random.uniform(3, 5))
                            break
                    except:
                        continue
                
                # Alternatif yöntem 2: URL'yi değiştir
                if not tab_found:
                    print("⚠️ Alternatif yöntem: URL üzerinden yorumlar sayfasına yönlendiriliyor...")
                    current_url = driver.current_url
                    # URL'ye yorumlar parametresi ekle
                    if "?" in current_url:
                        reviews_url = current_url + "&lrd=1"
                    else:
                        reviews_url = current_url + "?lrd=1"
                    driver.get(reviews_url)
                    time.sleep(random.uniform(3, 5))
        except Exception as e:
            print(f"⚠️ Yorumlar butonuna tıklama hatası: {e}")
        
        # Gerekli menü ayarları
        try:
            print("🔧 Menü ayarları yapılıyor...")
            
            # Sayfanın tamamen yüklenmesini bekle
            time.sleep(random.uniform(3, 5))
            
            # İlk butona tıkla
            try:
                print("🔍 İlk menü butonu aranıyor...")
                # CSS selector ile class'a göre ara (daha stabil)
                wait = WebDriverWait(driver, 10)
                first_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span.IAbLGd.google-symbols")))
                
                # Butonu görünür hale getir
                driver.execute_script("arguments[0].scrollIntoView(true);", first_button)
                time.sleep(1)
                
                # Tıkla
                driver.execute_script("arguments[0].click();", first_button)
                print("✅ İlk menü butonuna tıklandı")
                time.sleep(random.uniform(2, 3))
                
            except Exception as e:
                print(f"⚠️ İlk menü butonu tıklama hatası: {e}")
                # Alternatif yöntemler dene
                try:
                    print("🔄 Alternatif menü butonu aranıyor...")
                    
                    # XPath ile daha esnek arama
                    alt_xpaths = [
                        "//*[@id='QA0Szd']//button//span[@class='IAbLGd google-symbols']",
                        "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[10]/button/span",
                        "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[10]/button/span",
                        "//span[@class='IAbLGd google-symbols' and @aria-hidden='true']"
                    ]
                    
                    button_found = False
                    for xpath in alt_xpaths:
                        try:
                            alt_button = driver.find_element(By.XPATH, xpath)
                            if alt_button.is_displayed() and alt_button.is_enabled():
                                driver.execute_script("arguments[0].scrollIntoView(true);", alt_button)
                                time.sleep(0.5)
                                driver.execute_script("arguments[0].click();", alt_button)
                                print(f"✅ Alternatif menü butonuna tıklandı (XPath: {xpath})")
                                time.sleep(random.uniform(2, 3))
                                button_found = True
                                break
                        except:
                            continue
                    
                    if not button_found:
                        # CSS selector ile başka denemeler
                        alt_selectors = [
                            "button .IAbLGd.google-symbols",
                            "[data-value] span.google-symbols",
                            ".VfPpkd-Bz112c-LgbsSe span.google-symbols",
                            "button[role='button'] .google-symbols"
                        ]
                        
                        for selector in alt_selectors:
                            try:
                                alt_button = driver.find_element(By.CSS_SELECTOR, selector)
                                if alt_button.is_displayed():
                                    driver.execute_script("arguments[0].scrollIntoView(true);", alt_button)
                                    time.sleep(0.5)
                                    driver.execute_script("arguments[0].click();", alt_button)
                                    print(f"✅ Alternatif menü butonuna tıklandı (CSS: {selector})")
                                    time.sleep(random.uniform(2, 3))
                                    button_found = True
                                    break
                            except:
                                continue
                    
                    if not button_found:
                        print("⚠️ Hiçbir alternatif menü butonu bulunamadı")
                        
                except Exception as alt_error:
                    print(f"⚠️ Alternatif menü butonu arama hatası: {alt_error}")
            
            # Açılan menüden seçenek seç
            try:
                print("🔍 Menü seçeneği aranıyor...")
                # Menü açılması için biraz daha bekle
                time.sleep(random.uniform(1, 2))
                
                menu_option = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//*[@id='action-menu']/div[2]"))
                )
                
                # Seçeneği görünür hale getir
                driver.execute_script("arguments[0].scrollIntoView(true);", menu_option)
                time.sleep(0.5)
                
                # Tıkla
                driver.execute_script("arguments[0].click();", menu_option)
                print("✅ Menü seçeneği seçildi")
                time.sleep(random.uniform(3, 4))
                
            except Exception as e:
                print(f"⚠️ Menü seçeneği seçme hatası: {e}")
                # Alternatif seçenekler dene
                try:
                    print("🔄 Alternatif menü seçenekleri aranıyor...")
                    alt_options = driver.find_elements(By.CSS_SELECTOR, "#action-menu div, [role='menuitem'], .VfPpkd-StrnGf-rymPhb-ibnC6b")
                    if len(alt_options) >= 2:
                        driver.execute_script("arguments[0].click();", alt_options[1])
                        print("✅ Alternatif menü seçeneği seçildi")
                        time.sleep(random.uniform(3, 4))
                except:
                    print("⚠️ Alternatif menü seçeneği de bulunamadı")
                
        except Exception as e:
            print(f"⚠️ Menü ayarları genel hatası: {e}")
        
        # Menü işlemlerinden sonra ek bekleme
        print("⏳ Menü değişikliklerinin yüklenmesi bekleniyor...")
        time.sleep(random.uniform(3, 5))
        
        # Yorumları yüklemek için scroll yap
        print("📜 Tüm yorumlar için scroll yapılıyor...")
        previous_review_count = 0
        no_change_count = 0
        max_scroll_attempts = 30
        
        for scroll_attempt in range(max_scroll_attempts):
            # Mevcut yorumları say
            current_reviews = driver.find_elements(By.CSS_SELECTOR, ".wiI7pd")
            current_review_count = len(current_reviews)
            
            print(f"🔄 Scroll {scroll_attempt + 1}: {current_review_count}/{max_reviews} yorum bulundu")
            
            # Maksimum yorum sayısına ulaşıldıysa durdur
            if current_review_count >= max_reviews:
                print(f"✅ Maksimum {max_reviews} yorum sayısına ulaşıldı, scroll durduruldu")
                break
            
            # Scroll yap
            try:
                # Sayfanın sonuna kadar scroll
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(random.uniform(1, 2))
                
                # Farklı scroll container'ları dene
                scroll_containers = [
                    "[role='main'] [role='region']",
                    ".m6QErb.DxyBCb.kA9KIf.dS8AEf",
                    ".m6QErb",
                    ".section-layout.section-scrollbox"
                ]
                
                for container_selector in scroll_containers:
                    containers = driver.find_elements(By.CSS_SELECTOR, container_selector)
                    if containers:
                        for container in containers:
                            try:
                                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", container)
                                time.sleep(random.uniform(0.5, 1))
                            except:
                                pass
                        break
            except Exception as e:
                print(f"⚠️ Scroll hatası: {e}")
            
            # Değişim kontrolü
            if current_review_count == previous_review_count:
                no_change_count += 1
                print(f"⏳ Değişim yok ({no_change_count}/5)")
            else:
                no_change_count = 0
                previous_review_count = current_review_count
            
            # 5 kez üst üste değişim yoksa bitir
            if no_change_count >= 5:
                print(f"✅ Daha fazla yorum bulunamadı, toplam: {current_review_count}")
                break
            
            # "Daha fazla yorum göster" butonu varsa tıkla
            try:
                more_buttons = driver.find_elements(By.CSS_SELECTOR, ".w8nwRe.kyuRq")
                if more_buttons:
                    for button in more_buttons:
                        try:
                            if button.is_displayed():
                                driver.execute_script("arguments[0].click();", button)
                                print("🔄 'Daha fazla' butonuna tıklandı")
                                time.sleep(random.uniform(0.5, 1))
                        except:
                            pass
            except:
                pass
        
        # Her bir yorum kartını topla (her kart kullanıcı, puan, tarih ve varsa metni içerir)
        review_cards = driver.find_elements(By.CSS_SELECTOR, ".jftiEf")
        print(f"✅ Toplam {len(review_cards)} yorum kartı bulundu")

        # Maksimum sınır uygula
        if len(review_cards) > max_reviews:
            print(f"📊 İlk {max_reviews} yorum işlenecek")
            review_cards = review_cards[:max_reviews]

        # Her bir yorumu işle
        for i, card in enumerate(review_cards, 1):
            try:
                # Kart içindeki 'daha fazla' butonunu genişlet
                try:
                    more_btn = card.find_element(By.CSS_SELECTOR, ".w8nwRe.kyuRq")
                    if more_btn.is_displayed():
                        driver.execute_script("arguments[0].click();", more_btn)
                        time.sleep(random.uniform(0.3, 0.7))
                except:
                    pass

                # Kullanıcı adını al
                try:
                    user_elem = card.find_element(By.CSS_SELECTOR, ".d4r55")
                    username = user_elem.text.strip()
                    if not username:
                        username = user_elem.get_attribute("aria-label") or ""
                except:
                    username = ""

                # Tarihi al
                try:
                    date = card.find_element(By.CSS_SELECTOR, ".rsqaWe").text.strip()
                except:
                    date = ""

                # Puanı al
                try:
                    rating_elem = card.find_element(
                        By.CSS_SELECTOR,
                        ".DU9Pgb .kvMYJc, .DU9Pgb .kvMYYJc, .kvMYJc"  # yedek seçiciler
                    )
                    rating_text = rating_elem.get_attribute("aria-label") or ""
                    import re as _re
                    m = _re.search(r"([0-9]+(?:[\\.,][0-9]+)?)", rating_text)
                    rating = m.group(1).replace(",", ".") if m else ""
                except:
                    rating = ""

                # Yorum metnini al (varsa)
                actual_review = ""
                try:
                    comment_elem = card.find_element(By.CSS_SELECTOR, ".wiI7pd")
                    review_text = comment_elem.text.strip()

                    # Satır bazlı temizlik
                    lines = review_text.split('\n')

                    # Google bazen ilk satırı boş bırakabiliyor, temizle
                    if lines and not lines[0].strip():
                        lines = lines[1:]

                    # Son satır "Daha fazla" kalıntısıysa atla
                    if lines and ("daha fazla" in lines[-1].lower() or "more" in lines[-1].lower()):
                        lines = lines[:-1]

                    actual_review = "\n".join(lines).strip()
                except:
                    pass  # yorum metni yok (sadece puan)

                # Veri kaydet
                reviews.append({
                    "Yorum_Sırası": i,
                    "Kullanıcı Adı": username,
                    "Puan": rating,
                    "Yorum": actual_review,
                    "Yorum Tarihi": date
                })

                # İlerleme göster
                if i % 10 == 0:
                    print(f"⏳ {i}/{len(review_cards)} yorum işlendi...")

            except Exception as e:
                print(f"⚠️ Yorum {i} işlenirken hata: {e}")
        
        # Sekmeyi kapat ve ana sekmeye dön
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        # Yorumları CSV'ye kaydet
        if reviews:
            save_reviews_to_csv(reviews, place_name)
        
        return len(reviews)
        
    except Exception as e:
        print(f"❌ Yorum çıkarma hatası: {e}")
        # Hata durumunda ana sekmeye dön
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass
        return 0

def save_reviews_to_csv(reviews, place_name):
    """Yorumları CSV'ye kaydet"""
    if not reviews:
        print(f"❌ {place_name} için kaydedilecek yorum yok")
        return None
    
    # Dosya adını oluştur
    clean_name = clean_filename(place_name)
    filename = f"yorumlar_{clean_name}.csv"
    
    try:
        # DataFrame oluştur
        df = pd.DataFrame(reviews)
        
        # Klasör yoksa oluştur
        os.makedirs("yorumlar", exist_ok=True)
        
        # CSV'ye kaydet
        full_path = os.path.join("yorumlar", filename)
        df.to_csv(full_path, index=False, encoding='utf-8-sig', sep=';')
        
        print(f"💾 {len(reviews)} yorum '{full_path}' dosyasına kaydedildi")
        return full_path
        
    except Exception as e:
        print(f"❌ {place_name} yorum dosyası kaydetme hatası: {e}")
        return None

def main():
    """Ana fonksiyon - Sadece Yorumlar"""
    driver = None
    
    try:
        print("🔍 Google Maps Yorum Çıkarıcı")
        print("-" * 40)
        print("Bu program CSV dosyasındaki mekan linklerinden yorumları toplar")
        print("-" * 40)
        
        # CSV dosya adını al
        csv_filename = input("CSV dosya adını girin (örn: google_maps_restoranlar_detayli.csv): ").strip()
        
        if not csv_filename:
            print("❌ CSV dosya adı boş olamaz!")
            return
        
        if not os.path.exists(csv_filename):
            print(f"❌ '{csv_filename}' dosyası bulunamadı!")
            return
        
        # Maksimum yorum sayısını al
        try:
            max_reviews = int(input("Her mekan için kaç yorum toplanacak? (varsayılan: 100): ").strip() or "100")
        except:
            max_reviews = 100
        
        print(f"\n🚀 CSV'den mekan linkleri okunuyor...")
        places = read_csv_places(csv_filename)
        
        if not places:
            print("❌ CSV'de geçerli mekan linki bulunamadı!")
            return
        
        print(f"🌐 Chrome tarayıcı başlatılıyor...")
        driver = setup_driver()
        
        print(f"📊 {len(places)} mekan için yorumlar toplanmaya başlanıyor...")
        
        success_count = 0
        for i, place in enumerate(places, 1):
            try:
                print(f"\n{'='*60}")
                print(f"🔍 {i}/{len(places)}: {place['name']}")
                print(f"{'='*60}")
                
                review_count = extract_reviews(driver, place['link'], place['name'], max_reviews)
                
                if review_count > 0:
                    success_count += 1
                    print(f"✅ {place['name']} için {review_count} yorum başarıyla toplandı")
                else:
                    print(f"⚠️ {place['name']} için yorum alınamadı")
                
                # Rate limiting - mekanlar arası bekleme
                if i < len(places):
                    wait_time = random.uniform(2, 4)
                    print(f"⏳ Sonraki mekan için {wait_time:.1f} saniye bekleniyor...")
                    time.sleep(wait_time)
                    
            except Exception as e:
                print(f"❌ {place['name']} işlenirken hata: {e}")
                continue
        
        print(f"\n{'='*60}")
        print(f"✅ İşlem tamamlandı!")
        print(f"📊 Toplam {len(places)} mekan işlendi")
        print(f"✅ {success_count} mekan için yorumlar başarıyla toplandı")
        print(f"⚠️ {len(places) - success_count} mekan başarısız oldu")
        print(f"💾 Yorumlar 'yorumlar/' klasörüne kaydedildi")
        print(f"{'='*60}")
            
    except KeyboardInterrupt:
        print("\n⏹️ İşlem kullanıcı tarafından durduruldu")
        
    except Exception as e:
        print(f"❌ Genel hata: {str(e)}")
        
    finally:
        if driver:
            print("\n🔄 Tarayıcı kapatılıyor...")
            try:
                driver.quit()
                print("✅ İşlem tamamlandı")
            except:
                print("⚠️ Tarayıcı zaten kapalı")

if __name__ == "__main__":
    main() 

🔍 Google Maps Yorum Çıkarıcı
----------------------------------------
Bu program CSV dosyasındaki mekan linklerinden yorumları toplar
----------------------------------------

🚀 CSV'den mekan linkleri okunuyor...
✅ CSV'den 162 geçerli mekan linki okundu
🌐 Chrome tarayıcı başlatılıyor...
📊 162 mekan için yorumlar toplanmaya başlanıyor...

🔍 1/162: DENIZALTI Cafe & Restaurant

🔍 'DENIZALTI Cafe & Restaurant' için yorumlar toplanıyor (maksimum 100 yorum)...
✅ Yorumlar butonu bulundu: 'Yorumlar'
✅ Yorumlar butonuna tıklanıyor...
🔧 Menü ayarları yapılıyor...
🔍 İlk menü butonu aranıyor...
✅ İlk menü butonuna tıklandı
🔍 Menü seçeneği aranıyor...
✅ Menü seçeneği seçildi
⏳ Menü değişikliklerinin yüklenmesi bekleniyor...
📜 Tüm yorumlar için scroll yapılıyor...
🔄 Scroll 1: 6/100 yorum bulundu
🔄 'Daha fazla' butonuna tıklandı
🔄 'Daha fazla' butonuna tıklandı
🔄 'Daha fazla' butonuna tıklandı
🔄 'Daha fazla' butonuna tıklandı
🔄 'Daha fazla' butonuna tıklandı
🔄 'Daha fazla' butonuna tıklandı
🔄 'Daha f

In [1]:
import glob
import pandas as pd

In [4]:
csv_files = glob.glob('/Users/Serra/Desktop/Hulya_datas_term_project/*.csv')

fullData2 = pd.concat([pd.read_csv(file, delimiter=';') for file in csv_files], ignore_index=True)

In [5]:
fullData2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Restoran_Adi      379 non-null    object
 1   Adres             377 non-null    object
 2   Puan              379 non-null    object
 3   Yorum_Sayisi      379 non-null    object
 4   Kategori          379 non-null    object
 5   Fiyat_Araligi     379 non-null    object
 6   Google_Maps_Link  379 non-null    object
dtypes: object(7)
memory usage: 20.9+ KB


In [6]:
#FİLE ADINI YAZMA
import os

# Read each CSV and add a new column with the file name (without extension)
dfs = []
for file in csv_files:
    df = pd.read_csv(file, delimiter=';')
    # Extract file name without extension
    file_name = os.path.splitext(os.path.basename(file))[0]
    df['csv_file'] = file_name
    dfs.append(df)

fullData2 = pd.concat(dfs, ignore_index=True)

In [ ]:
#CSV'NIN TARİHİNİ EKLEME
import os
import datetime

# Read each CSV and add new columns: file name (without extension) and creation date
dfs = []
for file in csv_files:
    df = pd.read_csv(file, delimiter=';')
    file_name = os.path.splitext(os.path.basename(file))[0]
    # Get file creation time (as datetime)
    creation_ts = os.path.getctime(file)
    creation_date = datetime.datetime.fromtimestamp(creation_ts)
    df['csv_file'] = file_name
    df['csv_creation_date'] = creation_date
    dfs.append(df)

fullData3 = pd.concat(dfs, ignore_index=True)
fullData3

,Yorum_Sırası,Kullanıcı Adı,Puan,Yorum,Yorum Tarihi,csv_file,csv_creation_date
0,1,AREK KAZAR,NaN,Muazzam kibar ilgili çalışanlar. Yeşile Urla’n...,NaN,yorumlar_teruar_urla,2025-07-07 13:19:04.097325
1,2,Begul E,NaN,NaN,NaN,yorumlar_teruar_urla,2025-07-07 13:19:04.097325
2,3,mustafa belci,NaN,NaN,NaN,yorumlar_teruar_urla,2025-07-07 13:19:04.097325
3,4,Ruhsar Kucukgoz Emniyetli,NaN,Yemekleri beklendiği gibi ortalamanın bir hayl...,NaN,yorumlar_teruar_urla,2025-07-07 13:19:04.097325
4,5,U O,NaN,NaN,NaN,yorumlar_teruar_urla,2025-07-07 13:19:04.097325
...,...,...,...,...,...,...,...
8875,96,Nurettin TEN,5.0,Mezeleri çok lezzetli. Deniz ürünleride gayet ...,4 hafta önce,yorumlar_genc_balik_pisiricisi,2025-07-07 14:43:36.092699
8876,97,AZİZ ERCAN ÇELİKLİ,4.0,"Güzeldi, fiyatlarda makul.",4 hafta önce,yorumlar_genc_balik_pisiricisi,2025-07-07 14:43:36.092699
8877,98,İbrahim Oğuz Yörüdü,5.0,NaN,4 hafta önce,yorumlar_genc_balik_pisiricisi,2025-07-07 14:43:36.092699
8878,99,Evren Bölek,5.0,NaN,4 hafta önce,yorumlar_genc_balik_pisiricisi,2025-07-07 14:43:36.092699


In [15]:
import os

# Add a new column to each DataFrame with the total number of rows in its CSV
dfs = []
for file in csv_files:
    df = pd.read_csv(file, delimiter=';')
    file_name = os.path.splitext(os.path.basename(file))[0]
    row_count = len(df)
    df['csv_file'] = file_name
    df['csv_row_count'] = row_count
    dfs.append(df)

fullData2 = pd.concat(dfs, ignore_index=True)

In [10]:
fullData

,Yorum_Sırası,Kullanıcı Adı,Puan,Yorum,Yorum Tarihi,csv_file
0,1,AREK KAZAR,NaN,Muazzam kibar ilgili çalışanlar. Yeşile Urla’n...,NaN,yorumlar_teruar_urla
1,2,Begul E,NaN,NaN,NaN,yorumlar_teruar_urla
2,3,mustafa belci,NaN,NaN,NaN,yorumlar_teruar_urla
3,4,Ruhsar Kucukgoz Emniyetli,NaN,Yemekleri beklendiği gibi ortalamanın bir hayl...,NaN,yorumlar_teruar_urla
4,5,U O,NaN,NaN,NaN,yorumlar_teruar_urla
...,...,...,...,...,...,...
8875,96,Nurettin TEN,5.0,Mezeleri çok lezzetli. Deniz ürünleride gayet ...,4 hafta önce,yorumlar_genc_balik_pisiricisi
8876,97,AZİZ ERCAN ÇELİKLİ,4.0,"Güzeldi, fiyatlarda makul.",4 hafta önce,yorumlar_genc_balik_pisiricisi
8877,98,İbrahim Oğuz Yörüdü,5.0,NaN,4 hafta önce,yorumlar_genc_balik_pisiricisi
8878,99,Evren Bölek,5.0,NaN,4 hafta önce,yorumlar_genc_balik_pisiricisi


In [14]:
filtered_null_date = fullData[fullData["Yorum Tarihi"].isnull()]
filtered_null_date["csv_file"].unique()

array(['yorumlar_teruar_urla', 'yorumlar_urla_corbacisi_sanayi_subesi',
       'yorumlar_od_urla', 'yorumlar_istifci_urla', 'yorumlar_sulo_urla',
       'yorumlar_fermaki', 'yorumlar_urla_yakamoz_cafe_restoran'],
      dtype=object)

In [16]:
fullData2

,Yorum_Sırası,Kullanıcı Adı,Puan,Yorum,Yorum Tarihi,csv_file,csv_row_count
0,1,AREK KAZAR,NaN,Muazzam kibar ilgili çalışanlar. Yeşile Urla’n...,NaN,yorumlar_teruar_urla,100
1,2,Begul E,NaN,NaN,NaN,yorumlar_teruar_urla,100
2,3,mustafa belci,NaN,NaN,NaN,yorumlar_teruar_urla,100
3,4,Ruhsar Kucukgoz Emniyetli,NaN,Yemekleri beklendiği gibi ortalamanın bir hayl...,NaN,yorumlar_teruar_urla,100
4,5,U O,NaN,NaN,NaN,yorumlar_teruar_urla,100
...,...,...,...,...,...,...,...
8875,96,Nurettin TEN,5.0,Mezeleri çok lezzetli. Deniz ürünleride gayet ...,4 hafta önce,yorumlar_genc_balik_pisiricisi,100
8876,97,AZİZ ERCAN ÇELİKLİ,4.0,"Güzeldi, fiyatlarda makul.",4 hafta önce,yorumlar_genc_balik_pisiricisi,100
8877,98,İbrahim Oğuz Yörüdü,5.0,NaN,4 hafta önce,yorumlar_genc_balik_pisiricisi,100
8878,99,Evren Bölek,5.0,NaN,4 hafta önce,yorumlar_genc_balik_pisiricisi,100


In [20]:
fullData2["csv_file_clean"] = fullData2["csv_file"].str.replace(r"^yorumlar_", "", regex=True).str.replace("_", " ")
fullData2[["csv_file", "csv_file_clean"]].head()

,csv_file,csv_file_clean
0,yorumlar_teruar_urla,teruar urla
1,yorumlar_teruar_urla,teruar urla
2,yorumlar_teruar_urla,teruar urla
3,yorumlar_teruar_urla,teruar urla
4,yorumlar_teruar_urla,teruar urla


In [22]:
data = pd.read_csv('/Users/Serra/Desktop/google_maps_restoranlar_izmir_urla_detayli.csv', delimiter=';')
data

,Restoran_Adi,Adres,Puan,Yorum_Sayisi,Kategori,Fiyat_Araligi,Google_Maps_Link
0,OD Urla,NaN,'4.5,2831,Bilgi yok,·₺₺₺₺,https://www.google.com/maps/place/OD+Urla/data...
1,The Guru Urla,"Rüstem, Çiçek Sokak, 35430 Urla/İzmir",'4.6,851,Restoran,·₺1.000+,https://www.google.com/maps/place/The+Guru+Url...
2,Beğendik Abi,"Camiatik, Tatar Cami Sk. No:12, 35430 Urla/İzmir",'4.3,3168,Türk restoranı,·₺₺₺,https://www.google.com/maps/place/Be%C4%9Fendi...
3,Yengeç Restaurant,"İskele, 2121. Sk. No:6 D:1, 35430 Urla/İzmir",'4.3,1585,Restoran,·₺1.000+,https://www.google.com/maps/place/Yenge%C3%A7+...
4,Gia Urla Restoran,"Denizli, Harbiye Cd. No:13, 35430 Urla/İzmir",'4.6,523,Restoran,·₺1.000+,https://www.google.com/maps/place/Gia+Urla+Res...
...,...,...,...,...,...,...,...
115,Babil Pide & Lahmacun,Cami Atik Mah. M Mustafa Kemal Sahil Bulvarı N...,'4.7,157,Pideci,·₺100–200,https://www.google.com/maps/place/Babil+Pide+%...
116,French Burger Urla,"Camiatik, Tatar Cami Sk. No:18, 35430 Urla/İzmir",'3.9,61,Hamburger restoranı,·₺300–400,https://www.google.com/maps/place/French+Burge...
117,Urla Karadeniz,"Camianın Mahallesi, Malgaca Pazarı No:18, 3543...",'3.9,46,Aile restoranı,Bilgi yok,https://www.google.com/maps/place/Urla+Karaden...
118,Esnaftan Kağıtta Köfte,"Camiatik, Zafer Cd. No:25, 35430 Urla/İzmir",'4.8,89,Köfteci,·₺200–300,https://www.google.com/maps/place/Esnaftan+Ka%...


In [23]:
data["Restoran_Adi"] = data["Restoran_Adi"].str.lower()
data.head()

,Restoran_Adi,Adres,Puan,Yorum_Sayisi,Kategori,Fiyat_Araligi,Google_Maps_Link
0,od urla,NaN,'4.5,2831,Bilgi yok,·₺₺₺₺,https://www.google.com/maps/place/OD+Urla/data...
1,the guru urla,"Rüstem, Çiçek Sokak, 35430 Urla/İzmir",'4.6,851,Restoran,·₺1.000+,https://www.google.com/maps/place/The+Guru+Url...
2,beğendik abi,"Camiatik, Tatar Cami Sk. No:12, 35430 Urla/İzmir",'4.3,3168,Türk restoranı,·₺₺₺,https://www.google.com/maps/place/Be%C4%9Fendi...
3,yengeç restaurant,"İskele, 2121. Sk. No:6 D:1, 35430 Urla/İzmir",'4.3,1585,Restoran,·₺1.000+,https://www.google.com/maps/place/Yenge%C3%A7+...
4,gia urla restoran,"Denizli, Harbiye Cd. No:13, 35430 Urla/İzmir",'4.6,523,Restoran,·₺1.000+,https://www.google.com/maps/place/Gia+Urla+Res...


In [24]:
fullData2

,Yorum_Sırası,Kullanıcı Adı,Puan,Yorum,Yorum Tarihi,csv_file,csv_row_count,csv_file_clean
0,1,AREK KAZAR,NaN,Muazzam kibar ilgili çalışanlar. Yeşile Urla’n...,NaN,yorumlar_teruar_urla,100,teruar urla
1,2,Begul E,NaN,NaN,NaN,yorumlar_teruar_urla,100,teruar urla
2,3,mustafa belci,NaN,NaN,NaN,yorumlar_teruar_urla,100,teruar urla
3,4,Ruhsar Kucukgoz Emniyetli,NaN,Yemekleri beklendiği gibi ortalamanın bir hayl...,NaN,yorumlar_teruar_urla,100,teruar urla
4,5,U O,NaN,NaN,NaN,yorumlar_teruar_urla,100,teruar urla
...,...,...,...,...,...,...,...,...
8875,96,Nurettin TEN,5.0,Mezeleri çok lezzetli. Deniz ürünleride gayet ...,4 hafta önce,yorumlar_genc_balik_pisiricisi,100,genc balik pisiricisi
8876,97,AZİZ ERCAN ÇELİKLİ,4.0,"Güzeldi, fiyatlarda makul.",4 hafta önce,yorumlar_genc_balik_pisiricisi,100,genc balik pisiricisi
8877,98,İbrahim Oğuz Yörüdü,5.0,NaN,4 hafta önce,yorumlar_genc_balik_pisiricisi,100,genc balik pisiricisi
8878,99,Evren Bölek,5.0,NaN,4 hafta önce,yorumlar_genc_balik_pisiricisi,100,genc balik pisiricisi


In [25]:
# Merge fullData2 and data on cleaned restaurant names
merged = fullData2.merge(
    data,
    left_on="csv_file_clean",
    right_on="Restoran_Adi",
    how="left",
    suffixes=('', '_restoran')
)
merged.head()

,Yorum_Sırası,Kullanıcı Adı,Puan,Yorum,Yorum Tarihi,csv_file,csv_row_count,csv_file_clean,Restoran_Adi,Adres,Puan_restoran,Yorum_Sayisi,Kategori,Fiyat_Araligi,Google_Maps_Link
0,1,AREK KAZAR,NaN,Muazzam kibar ilgili çalışanlar. Yeşile Urla’n...,NaN,yorumlar_teruar_urla,100,teruar urla,teruar urla,"Kuşçular, 8028. Sk. No:16, 35430 Urla/İzmir",'4.4,324.0,Web sitesi ekleyin,₺16.455,https://www.google.com/maps/place/TERUAR+URLA/...
1,2,Begul E,NaN,NaN,NaN,yorumlar_teruar_urla,100,teruar urla,teruar urla,"Kuşçular, 8028. Sk. No:16, 35430 Urla/İzmir",'4.4,324.0,Web sitesi ekleyin,₺16.455,https://www.google.com/maps/place/TERUAR+URLA/...
2,3,mustafa belci,NaN,NaN,NaN,yorumlar_teruar_urla,100,teruar urla,teruar urla,"Kuşçular, 8028. Sk. No:16, 35430 Urla/İzmir",'4.4,324.0,Web sitesi ekleyin,₺16.455,https://www.google.com/maps/place/TERUAR+URLA/...
3,4,Ruhsar Kucukgoz Emniyetli,NaN,Yemekleri beklendiği gibi ortalamanın bir hayl...,NaN,yorumlar_teruar_urla,100,teruar urla,teruar urla,"Kuşçular, 8028. Sk. No:16, 35430 Urla/İzmir",'4.4,324.0,Web sitesi ekleyin,₺16.455,https://www.google.com/maps/place/TERUAR+URLA/...
4,5,U O,NaN,NaN,NaN,yorumlar_teruar_urla,100,teruar urla,teruar urla,"Kuşçular, 8028. Sk. No:16, 35430 Urla/İzmir",'4.4,324.0,Web sitesi ekleyin,₺16.455,https://www.google.com/maps/place/TERUAR+URLA/...


In [29]:
deneme = (merged[(merged["csv_row_count"] < 100) & (merged["Yorum_Sayisi"] > 100)])

In [31]:
deneme["csv_file_clean"].unique()

array(['aslinda meyhane urla', 'motto garden urla',
       'girit pastanesi 1950', 'od urla', 'ceviz garden urla',
       'the guru urla', 'adresim urla restoran', 'pizza burger paletto',
       'wine house urla', 'fermaki', 'urla yakamoz cafe restoran',
       'noon good food'], dtype=object)

In [3]:
import glob
import pandas as pd
csv_files = glob.glob('/Users/Serra/Desktop/yorumlar/*.csv')
yorumlar = pd.concat([pd.read_csv(file, delimiter=';') for file in csv_files], ignore_index=True)

In [7]:
yorumlar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117448 entries, 0 to 117447
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Yorum_Sırası   117448 non-null  int64 
 1   Kullanıcı Adı  117447 non-null  object
 2   Puan           117448 non-null  int64 
 3   Yorum          78890 non-null   object
 4   Yorum Tarihi   117448 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.5+ MB


In [9]:
yorumlar_filtered = yorumlar.dropna(subset=['Yorum']).copy()
yorumlar_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78890 entries, 0 to 117447
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Yorum_Sırası   78890 non-null  int64 
 1   Kullanıcı Adı  78889 non-null  object
 2   Puan           78890 non-null  int64 
 3   Yorum          78890 non-null  object
 4   Yorum Tarihi   78890 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.6+ MB


In [15]:
import glob
import pandas as pd
restoran_files = glob.glob('/Users/Serra/Desktop/restoranlar/*.csv')
restoranlar = pd.concat([pd.read_csv(file, delimiter=';') for file in restoran_files], ignore_index=True)
restoranlar

,Restoran_Adi,Adres,Puan,Yorum_Sayisi,Kategori,Fiyat_Araligi,Google_Maps_Link
0,OD Urla,NaN,'4.5,2831,Bilgi yok,·₺₺₺₺,https://www.google.com/maps/place/OD+Urla/data...
1,The Guru Urla,"Rüstem, Çiçek Sokak, 35430 Urla/İzmir",'4.6,851,Restoran,·₺1.000+,https://www.google.com/maps/place/The+Guru+Url...
2,Beğendik Abi,"Camiatik, Tatar Cami Sk. No:12, 35430 Urla/İzmir",'4.3,3168,Türk restoranı,·₺₺₺,https://www.google.com/maps/place/Be%C4%9Fendi...
3,Yengeç Restaurant,"İskele, 2121. Sk. No:6 D:1, 35430 Urla/İzmir",'4.3,1585,Restoran,·₺1.000+,https://www.google.com/maps/place/Yenge%C3%A7+...
4,Gia Urla Restoran,"Denizli, Harbiye Cd. No:13, 35430 Urla/İzmir",'4.6,523,Restoran,·₺1.000+,https://www.google.com/maps/place/Gia+Urla+Res...
...,...,...,...,...,...,...,...
947,Fatih Pide,"İsmet İnönü, 2030. Sk. 11 A, 35937 Çeşme/İzmir",'4.4,184,Pideci,·₺200–300,https://www.google.com/maps/place/Fatih+Pide/d...
948,Element Restaurant,"Radisson Blu Resort & Spa, Altınyunus, 3435. S...",'3.0,232,Restoran,Bilgi yok,https://www.google.com/maps/place/Element+Rest...
949,Çeşme Aton Kokorec Midye,"3053 sok 16 Eylül Mahallesi 28/a, 35930 Çeşme/...",'3.3,37,Restoran,Bilgi yok,https://www.google.com/maps/place/%C3%87e%C5%9...
950,Çeşme Izgara Evi ( Grill House ),"16 Eylül, 3001. Sk., 35930 Çeşme/İzmir",'4.3,232,Izgara,Bilgi yok,https://www.google.com/maps/place/%C3%87e%C5%9...


In [17]:
restoranlar["Yorum_Sayisi"].describe()

count      952.000000
mean       689.510504
std       1950.952905
min         11.000000
25%         87.750000
50%        262.000000
75%        641.500000
max      33140.000000
Name: Yorum_Sayisi, dtype: float64

ALAKASIZ EKLER

In [ ]:
category_mapping = {
    "Restoran": "Genel",
    "Türk restoranı": "Türk Mutfağı",
    "Balık Restoranı": "Deniz Ürünleri",
    "Kahvaltı restoranı": "Kahvaltı",
    "Kafe": "Kafe",
    "Meyhane": "Meyhane",
    "Deniz mahsülleri restoranı": "Deniz Ürünleri",
    "Dönerci": "Türk Mutfağı",
    "Tatlıcı": "Tatlı & Pastane",
    "Fast food lokantası": "Fast Food",
    "Pideci": "Türk Mutfağı",
    "Pizza restoranı": "İtalyan Mutfağı",
    "Kebap Restoranı": "Türk Mutfağı",
    "Hamburger restoranı": "Fast Food",
    "Bilgi yok": "Bilinmiyor",
    "Et Lokantası": "Türk Mutfağı",
    "Büfe": "Fast Food",
    "Aile restoranı": "Genel",
    "İtalyan restoranı": "İtalyan Mutfağı",
    "Çiğ Köfteci": "Türk Mutfağı",
    "Köfteci": "Türk Mutfağı",
    "Dürüm Restoranı": "Türk Mutfağı",
}

df["Kategori"] = df["Kategori"].map(category_mapping).fillna(df["Kategori"])

# Kategori sayılarına tekrar bakalım
print(df["Kategori"].value_counts())


In [ ]:
# Silinecek kategoriler
invalid_categories = [
    "Bilgi yok",
    "Bilinmiyor",
    "Web sitesi ekleyin",
    "Düğün Planlamacısı",
    "Değirmen",
    "Tekne Gezisi Acentesi"
]

# Bu kategorilere sahip satırları düşür
df = df[~df["Kategori"].isin(invalid_categories)].copy()

# Fiyat Aralığını High-Medium-Low yapmak

In [ ]:
import re
import numpy

def classify_price(value):
    if pandas.isna(value):
        return numpy.nan

    # Strip whitespace and common garbage characters
    value = value.strip()
    value = value.replace("·", "").replace("•", "").replace("*", "").strip()

    # Handle symbolic indicators (₺, ₺₺, ₺₺₺, etc.)
    if re.fullmatch(r"₺{1}", value):
        return "Low"
    elif re.fullmatch(r"₺{2}", value):
        return "Medium"
    elif re.fullmatch(r"₺{3,}", value):
        return "High"

    # Handle explicit "Bilgi yok" or unknown strings
    if "bilgi yok" in value.lower():
        return numpy.nan

    # Clean up symbols from range-based entries
    clean = value.replace("₺", "").replace(".", "").replace("...", "").replace(" ", "").strip()
    clean = re.sub(r"[–—−]", "-", clean)  # normalize various dash types

    # Handle ranges like "100-200"
    if "-" in clean:
        try:
            min_val = int(clean.split("-")[0])
        except ValueError:
            return numpy.nan
    elif "+" in clean:
        try:
            min_val = int(clean.replace("+", ""))
        except ValueError:
            return numpy.nan
    else:
        return numpy.nan

    # Classify
    if min_val < 200:
        return "Low"
    elif min_val < 500:
        return "Medium"
    else:
        return "High"

# Apply to your dataframe
df["Fiyat_Araligi"] = df["Fiyat_Araligi"].apply(classify_price)
df

# İL VE İLÇE OLARAK AYIRMAK

In [ ]:
import pandas as pd
import re

def extract_ilce_il(adres):
    try:
        # Find the last part after the last comma
        last_part = adres.split(',')[-1].strip()

        if '/' in last_part:
            ilce_raw, il_raw = [x.strip() for x in last_part.split('/')]
        else:
            # If no slash, fall back to space-separated parts
            parts = last_part.split()
            il_raw = parts[-1]
            ilce_raw = ' '.join(parts[:-1])

        # Remove postal code (e.g. 35430) from ilçe
        ilce = re.sub(r"^\d{4,5}\s*", "", ilce_raw)

        return pd.Series({'İlçe': ilce, 'İl': il_raw})
    except Exception:
        return pd.Series({'İlçe': None, 'İl': None})

# Uygula
df[['İlçe', 'İl']] = df['Adres'].apply(extract_ilce_il)
df.drop(columns=['Adres'], inplace=True)
df


In [ ]:
df = df.drop_duplicates(subset=["Restoran_Adi", "Yorum_Sayisi", "İlçe"])
df